### 필요한 라이브러리 로드

In [2]:
from ultralytics import YOLO
import mediapipe as mp
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader, random_split
import cv2
import numpy as np
import os
import random
import platform
import yaml
from tqdm import tqdm
import gc
from torch.cuda import empty_cache

if platform.system() == "Darwin":
    print("your system is mac os")
    device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
else:
    print("your system is cuda")
    device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

your system is cuda


In [2]:
data_dir = '/home/pepsi/dev_ws/mldl/Training/datasets/person'
model_name = "yolov8n.pt"

classes = ['person']
nc = len(classes)
yaml_file = 'data.yaml'

In [3]:
data = {
    'train' : data_dir + '/train/',
    'val' : data_dir + '/valid/',
    'test' : data_dir + '/test/',
    'nc' : nc,
    'names' : classes,
}

In [4]:
os.makedirs(data_dir, exist_ok=True)

with open(data_dir + '/' + yaml_file, 'wt') as fw:
    yaml.dump(data, fw)

In [5]:
with open(data_dir + '/' + yaml_file, 'rt') as fr:
    d = yaml.safe_load(fr)
    print(type(d))
    print(d)

<class 'dict'>
{'names': ['person'], 'nc': 1, 'test': '/home/pepsi/dev_ws/mldl/Training/datasets/person/test/', 'train': '/home/pepsi/dev_ws/mldl/Training/datasets/person/train/', 'val': '/home/pepsi/dev_ws/mldl/Training/datasets/person/valid/'}


In [ ]:
train_epoch = 100
patience = 30
batch = 32
imgsz = 640
LR = 0.001
optimizer = 'AdamW'

In [ ]:
yolo_model = YOLO(model_name).to(device)

yolo_model.train(data=data_dir + '/' + yaml_file,
            epochs = train_epoch,
            patience=patience,
            batch=batch,
            imgsz = imgsz,
            optimizer=optimizer)

In [ ]:
YOLO_best_model_path = '/Users/wjsong/dev_ws/Hosbot/runs/detect/train/weights/best.pt'

YOLO_best_model = YOLO(YOLO_best_model_path).to(device)
metrics = YOLO_best_model.val()

for label, ap in zip(classes, metrics.box.maps):
    print(label, ':', ap)

In [3]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

yolo_best_model = '/home/pepsi/dev_ws/mldl/Training/runs/detect/train3/weights/best.pt'
yolo_model = YOLO(yolo_best_model).to(device)

In [4]:
def get_YOLO_box(img, yolo_model, detect_cls):
    box_results = yolo_model.predict(img, conf=0.6, verbose=False, show=False)
    box_results = box_results[0].boxes

    boxes = box_results.xyxy.cpu().tolist()
    box_class = box_results.cls.cpu().tolist()

    p1x1, p1y1, p1x2, p1y2 = 0, 0, 0, 0
    p2x1, p2y1, p2x2, p2y2 = 0, 0, 0, 0
    for idx, cls in enumerate(box_class):
        if int(cls) == detect_cls:
            p1x1, p1y1, p1x2, p1y2 = boxes[0]
            p1x1, p1y1, p1x2, p1y2 = int(p1x1), int(p1y1), int(p1x2), int(p1y2)

            if len(boxes) > 1:
                p2x1, p2y1, p2x2, p2y2 = boxes[1]
                p2x1, p2y1, p2x2, p2y2 = int(p2x1), int(p2y1), int(p2x2), int(p2y2)

    return p1x1, p1y1, p1x2, p1y2, p2x1, p2y1, p2x2, p2y2

In [5]:
def get_pose_landmarks(results):
    xyz_list = []
    for landmark in results.pose_landmarks.landmark:
        xyz_list.append(landmark.x)
        xyz_list.append(landmark.y)
        xyz_list.append(landmark.z)
    return xyz_list 

In [6]:
def append_data_xyz_list_list(xyz_list_list, p1x1, p1y1, p1x2, p1y2, p2x1, p2y1, p2x2, p2y2, xyz_list):
    xyz_list.append(abs(p1x1 - p2x1) / 640)
    xyz_list.append(abs(p1x2 - p2x2) / 640)
    xyz_list.append(abs(p1y1 - p2y1) / 640)
    xyz_list.append(abs(p1y2 - p2y2) / 640)
    xyz_list_list.append(xyz_list)

    return xyz_list_list

In [7]:
def generate_dataset(mp_pose, video_path, detect_cls):
    xyz_list_list = []
    poses = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

    cap = cv2.VideoCapture(video_path)

    if cap.isOpened():
        while True:
            ret, img = cap.read()
            if ret == True:
                xyz_list = []
                img = cv2.resize(img, (640, 640))
                results = poses.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

                if not results.pose_landmarks: continue

                xyz_list = get_pose_landmarks(results)
                p1x1, p1y1, p1x2, p1y2, p2x1, p2y1, p2x2, p2y2 = get_YOLO_box(img, yolo_model, detect_cls)

                if (p1x1 == 0 and p1y1 == 0 and p1x2 == 0 and p1y2== 0) and (p2x1 == 0 and p2y1 == 0 and p2x2 == 0 and p2y2== 0): continue

                xyz_list_list = append_data_xyz_list_list(xyz_list_list, p1x1, p1y1, p1x2, p1y2, p2x1, p2y1, p2x2, p2y2, xyz_list)

                cv2.waitKey(1)
            else:
                break

    cap.release()
    gc.collect()
    torch.cuda.empty_cache()
    
    return xyz_list_list

In [96]:
Video_path = './datasets/pose/train2'
video_name_list = os.listdir(Video_path)
dataset = []
length = 18
detect_cls = 0

for video_name in tqdm(video_name_list):
    if 'normal' in video_name: label = 0
    elif 'fighting' in video_name: label = 1
    elif 'lying' in video_name: label = 2
    elif 'smoking' in video_name: label = 3

    pose_data = generate_dataset(mp_pose, '{}/{}'.format(Video_path, video_name), detect_cls)

    for idx in range(0, len(pose_data), int(length)):
        seq_list = pose_data[idx : idx + length]
        if len(seq_list) == length:
            dataset.append({'key' : label, 'value': seq_list})

random.shuffle(dataset)

  0%|          | 0/16 [00:00<?, ?it/s]

I0000 00:00:1743662616.421273   12532 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743662616.425521   14915 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1743662616.524199   14902 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743662616.564774   14909 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
  6%|▋         | 1/16 [00:30<07:44, 30.96s/it]I0000 00:00:1743662647.379519   12532 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743662647.381604   14947 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1743662647.502666   14935 in

In [97]:
print(len(dataset))

261


In [98]:
print('input data', dataset[0]['value'][0])
print('input data length', len(dataset[0]['value'][0]))

input data [0.4896206259727478, 0.2955930233001709, -0.18025879561901093, 0.4944857954978943, 0.2864210605621338, -0.16384553909301758, 0.49717432260513306, 0.28647130727767944, -0.16392391920089722, 0.49956101179122925, 0.2866463363170624, -0.16397570073604584, 0.48687291145324707, 0.2858287990093231, -0.16399261355400085, 0.484457790851593, 0.28549468517303467, -0.16400600969791412, 0.48215824365615845, 0.28534311056137085, -0.164031520485878, 0.5022401809692383, 0.28879401087760925, -0.06746762245893478, 0.4802505373954773, 0.2882291376590729, -0.06947915256023407, 0.49430474638938904, 0.3035470247268677, -0.1443376988172531, 0.4861088693141937, 0.3035762906074524, -0.1451880782842636, 0.5216508507728577, 0.34751608967781067, -0.033903468400239944, 0.46449166536331177, 0.34376296401023865, -0.025703245773911476, 0.526207447052002, 0.41610851883888245, -0.0362040139734745, 0.45846644043922424, 0.39264366030693054, -0.15504685044288635, 0.5269924998283386, 0.47789448499679565, -0.0610

In [99]:
class MyDataset(Dataset):
    def __init__(self, seq_list):
        self.X = []
        self.y = []
        for dic in seq_list:
            self.y.append(dic['key'])
            self.X.append(dic['value'])

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))

In [100]:
split_ratio = [0.7, 0.2, 0.1]
train_len = int(len(dataset) * split_ratio[0])
val_len = int(len(dataset) * split_ratio[1])
test_len = len(dataset) - train_len - val_len

In [101]:
train_dataset = MyDataset(dataset)
train_data, valid_data, test_data = random_split(train_dataset, [train_len, val_len, test_len])

train_loader = DataLoader(train_data, batch_size=8)
val_loader = DataLoader(valid_data, batch_size=8)
test_loader = DataLoader(test_data, batch_size=8)

In [102]:
class LSTM(nn.Module):
    def __init__(self, num_layers=1):
        super(LSTM, self).__init__()
        self.lstm1 = nn.LSTM(103, 128, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm1 = nn.LayerNorm(256)
        self.dropout1 = nn.Dropout(0.1)

        self.lstm2 = nn.LSTM(256, 64, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm2 = nn.LayerNorm(128)
        self.dropout2 = nn.Dropout(0.1)

        self.lstm3 = nn.LSTM(128, 32, num_layers, batch_first=True, bidirectional=True)
        self.layer_norm3 = nn.LayerNorm(64)
        self.dropout3 = nn.Dropout(0.1)

        self.attention = nn.Linear(64, 1)
        self.fc = nn.Linear(64, 4)


    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.layer_norm1(x)
        x = self.dropout1(x)

        x, _ = self.lstm2(x)
        x = self.layer_norm2(x)
        x = self.dropout2(x)

        x, _ = self.lstm3(x)
        x = self.layer_norm3(x)
        x = self.dropout3(x)

        attention_weights = torch.softmax(self.attention(x), dim=1)
        x = torch.sum(attention_weights * x, dim=1)

        x = self.fc(x)
        return x

In [103]:
def init_model():
    global net, loss_fn, optim
    net = LSTM().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optim = AdamW(net.parameters(), lr = 0.001)

In [104]:
def init_epoch():
    global epoch_cnt
    epoch_cnt = 0

In [105]:
def init_log():
    global iter_log, tloss_log, tacc_log, vloss_log, vacc_log, log_stack, time_log
    iter_log, tloss_log, tacc_log, vloss_log, vacc_log = [], [], [], [], []
    log_stack, time_log = [], []

In [106]:
def record_train_log(_tloss, _tacc, _time):
    time_log.append(_time)
    tloss_log.append(_tloss)
    tacc_log.append(_tacc)
    iter_log.append(epoch_cnt)

In [107]:
def record_valid_log(_vloss, _vacc):
    vloss_log.append(_vloss)
    vacc_log.append(_vacc)

In [108]:
def last(log_list):
    if len(log_list) > 0:
        return log_list[len(log_list) - 1]
    else:
        return -1

In [109]:
def print_log():
    train_loss = round(float(last(tloss_log)), 3)
    train_acc = round(float(last(tacc_log)), 3)
    val_loss = round(float(last(vloss_log)), 3)
    val_acc = round(float(last(vacc_log)), 3)
    time_spent = round(float(last(time_log)), 3)

    log_str = 'Epoch: {:3}| T_Loss {:5} | T_acc {:5}| V_Loss {:5}| V_acc {:5} | {:5}'.format(last(iter_log), train_loss, train_acc, val_loss, val_acc, time_spent)

    log_stack.append(log_str) #프린트 준비

    for idx in reversed(range(len(log_stack))):
        print(log_stack[idx])

In [110]:
def clear_memory():
    if device != 'cpu':
        empty_cache()
    gc.collect()
    
def epoch_not_finished():
    return epoch_cnt < maximum_epoch

In [111]:
def epoch(data_loader, mode='train'):
    global epoch_cnt

    iter_loss, iter_acc, last_grad_performed = [], [], False

    for _data, _label in data_loader:
        data, label = _data.to(device), _label.type(torch.LongTensor).to(device)

        if mode == 'train' : net.train()
        else: net.eval()

        result = net(data)
        _, out = torch.max(result, 1)

        loss = loss_fn(result, label)
        iter_loss.append(loss.item())

        if mode == 'train':
            optim.zero_grad()
            loss.backward()
            optim.step()
            last_grad_performed = True

        acc_partial = (out == label).float().sum()
        acc_partial = acc_partial / len(label)
        iter_acc.append(acc_partial.item())


    if last_grad_performed:
        epoch_cnt += 1

    clear_memory()

    return np.average(iter_loss), np.average(iter_acc)

In [112]:
import time

init_model()
init_epoch()
init_log()
maximum_epoch = 50

In [113]:
while epoch_not_finished():
    start_time = time.time()
    tloss, tacc = epoch(train_loader, mode='train')
    end_time = time.time()
    time_taken = end_time - start_time
    record_train_log(tloss, tacc, time_taken)
    with torch.no_grad():
        vloss, vacc = epoch(val_loader, mode= 'val')
        record_valid_log(vloss, vacc)
    print_log()

print('\n Training completed!')

Epoch:   1| T_Loss 0.986 | T_acc 0.524| V_Loss 0.703| V_acc 0.661 | 0.855
Epoch:   2| T_Loss 0.649 | T_acc 0.703| V_Loss 0.385| V_acc 0.857 | 0.917
Epoch:   1| T_Loss 0.986 | T_acc 0.524| V_Loss 0.703| V_acc 0.661 | 0.855
Epoch:   3| T_Loss 0.434 | T_acc 0.781| V_Loss 0.269| V_acc 0.929 | 0.906
Epoch:   2| T_Loss 0.649 | T_acc 0.703| V_Loss 0.385| V_acc 0.857 | 0.917
Epoch:   1| T_Loss 0.986 | T_acc 0.524| V_Loss 0.703| V_acc 0.661 | 0.855
Epoch:   4| T_Loss 0.309 | T_acc 0.897| V_Loss 0.211| V_acc 0.911 | 1.035
Epoch:   3| T_Loss 0.434 | T_acc 0.781| V_Loss 0.269| V_acc 0.929 | 0.906
Epoch:   2| T_Loss 0.649 | T_acc 0.703| V_Loss 0.385| V_acc 0.857 | 0.917
Epoch:   1| T_Loss 0.986 | T_acc 0.524| V_Loss 0.703| V_acc 0.661 | 0.855
Epoch:   5| T_Loss 0.342 | T_acc 0.862| V_Loss 0.143| V_acc 0.982 | 1.055
Epoch:   4| T_Loss 0.309 | T_acc 0.897| V_Loss 0.211| V_acc 0.911 | 1.035
Epoch:   3| T_Loss 0.434 | T_acc 0.781| V_Loss 0.269| V_acc 0.929 | 0.906
Epoch:   2| T_Loss 0.649 | T_acc 0.703

In [114]:
with torch.no_grad():
    test_loss, test_acc = epoch(test_loader, mode='test')
    test_acc = round(test_acc, 4)
    test_loss = round(test_loss, 4)
    print('Test Acc: {}'.format(test_acc))
    print('Test Loss: {}'.format(test_loss))

Test Acc: 1.0
Test Loss: 0.0017


In [115]:
model_path = './models/lstm_model.pth'

torch.save(net.state_dict(), model_path)
print('모델이 저장되었습니다.')

모델이 저장되었습니다.


In [116]:
model_path = './models/lstm_model.pth'

lstm_model = LSTM().to(device)
lstm_model.load_state_dict(torch.load(model_path, map_location=device))
lstm_model.eval()
print("모델이 성공적으로 불러와졌습니다.")

모델이 성공적으로 불러와졌습니다.


In [117]:
length = 18
detect_cls = 0

lstm_model.eval()
dataset = []
status = 'None'

mp_pose = mp.solutions.pose
poses = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

xyz_list_list = []
status_dict = {0: 'normal', 1: 'fighting', 2: 'lying', 3: 'smoking'}

I0000 00:00:1743663130.583804   12532 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743663130.587448   15527 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)


W0000 00:00:1743663130.669712   15519 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743663130.714371   15523 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [118]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    RuntimeError("카메라 열기 실패")

while True:
    ret, frame = cap.read()
    if not ret:  # 프레임 읽기 실패 시 종료
        break

    # 프레임 크기 조정 (선택 사항, YOLO 모델에 따라 필요)
    frame = cv2.resize(frame, (640, 640))

    # Mediapipe 포즈 추출
    results = poses.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    xyz_list = []

    if results.pose_landmarks:

        # 포즈 랜드마크 추출 및 그리기
        for landmark in results.pose_landmarks.landmark:
            xyz_list.append(landmark.x)
            xyz_list.append(landmark.y)
            xyz_list.append(landmark.z)

        # YOLO 박스 예측
        box_results = yolo_model.predict(frame, conf=0.6, verbose=False, show=False)[0].boxes
        boxes = box_results.xyxy.cpu().tolist()
        box_class = box_results.cls.cpu().tolist()

        p1x1, p1y1, p1x2, p1y2 = 0, 0, 0, 0
        p2x1, p2y1, p2x2, p2y2 = 0, 0, 0, 0
        for idx, cls in enumerate(box_class):
            if int(cls) == detect_cls:
                p1x1, p1y1, p1x2, p1y2 = map(int, boxes[0])
                if len(boxes) > 1:
                    p2x1, p2y1, p2x2, p2y2 = map(int, boxes[1])

                break

        # YOLO 박스 좌표 정규화 후 추가
        xyz_list.extend([abs(p1x1 - p2x1) / 640, abs(p1x2 - p2x2) / 640, abs(p1y1 - p2y1) / 640, abs(p1y2 - p2y2) / 640])
        xyz_list_list.append(xyz_list)

    # 시퀀스 길이에 도달하면 LSTM 예측 수행
    if len(xyz_list_list) == length:
        dataset = [{'key': 0, 'value': xyz_list_list}]  # 임시 라벨 0 사용
        dataset = MyDataset(dataset)
        dataset_loader = DataLoader(dataset, batch_size=1)

        for data, _ in dataset_loader:
            data = data.to(device)
            with torch.no_grad():
                result = lstm_model(data)
                _, out = torch.max(result, 1)
                status = status_dict.get(out.item(), 'Unknown')

        xyz_list_list = []  # 시퀀스 초기화

    # 상태 텍스트 표시
    cv2.putText(frame, status, (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (255, 0, 0), 2)

    # 프레임 표시
    cv2.imshow('frame', frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()